# Submission File - Score 0.410856 Private

This is the submission file to submit the models built to the [Kaggle Competition](https://www.kaggle.com/competitions/tlvmc-parkinsons-freezing-gait-prediction). I have to thank Google for Tensorflow and the GPU usuage for training the models and [Baurzhan Urazalinov](https://www.kaggle.com/baurzhanurazalinov), who helped build and design these models in tensorflow.

The submission code consists of 3 parts: 
1. **Tdcsfog Model**: Here we define the tdcsfog model, then load tdcsfog test data and predict targets. Located at `/kaggle/input/defog-freezing-gait-models/025_0.355_0.816_0.1436_model.h5`
2. **Defog Model**: Here we define the defog model, then load defog test data and predict targets. Located at `/kaggle/input/tfog-freezing-gait-models/016_0.557_0.893_0.0798_model.h5`
3. Submission: Here the predicted values are collected, uniformly averaged (if several models were used), then the submission.csv is created.

Both of these models where built and saved using Google's GPU P100 availble on Kaggle for this competition. You can find the saved models in `/kaggle/input/tfog-freezing-gait-models` and `/kaggle/input/defog-freezing-gait-models`.

# Import Libraries and Utilities

We will import the files and utility files.

In [ ]:
import os 
import math

import numpy as np 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm
from joblib import Parallel, delayed

### Files
Here we import the test files we will need to run predicitions on. They are in seperate locations for tdcsfog and defog.

In [2]:
all_submissions = [] 

tsfog_ids = [fname.split('.')[0] for fname in os.listdir('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/tdcsfog')] 
defog_ids = [fname.split('.')[0] for fname in os.listdir('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/defog')] 

## Functions

First we will set up the parameters for our Tfog model. These are the same parameters used in the training file located within this folder.

Just like before we are going to define some functions. 

1. `sample_normalize` is a function that we use to take one sample(file in this case) and standardize the values around 0 for each accelometer column (AccV, AccML, AccAP).
2. `get_blocks` is our function to input series values into our model in blocks. First the block is padded is it divisible by `CFG['block_size']`. Then it gets the blocks by `CFG['block_stride']`, returning the blocks in that order. So the first block is 'series[0:15552, :]', second block is 'series[972:16524, :]' so on, so forth. These batch sizes are set by the GPU.


In [3]:
CFG = {'TPU': 0,
       'block_size': 15552, 
       'block_stride': 15552//16,
       'patch_size': 18, 
       
       'fog_model_dim': 320,
       'fog_model_num_heads': 6,
       'fog_model_num_encoder_layers': 5,
       'fog_model_num_lstm_layers': 2,
       'fog_model_first_dropout': 0.1,
       'fog_model_encoder_dropout': 0.1,
       'fog_model_mha_dropout': 0.0,
      }

assert CFG['block_size'] % CFG['patch_size'] == 0
assert CFG['block_size'] % CFG['block_stride'] == 0

def sample_normalize(sample):
    mean = tf.math.reduce_mean(sample)
    std = tf.math.reduce_std(sample)
    sample = tf.math.divide_no_nan(sample-mean, std)
    
    return sample.numpy()

def get_blocks(series, columns):
    series = series.copy()
    series = series[columns]
    series = series.values
    series = series.astype(np.float32)
    
    block_count = math.ceil(len(series) / CFG['block_size'])
    
    series = np.pad(series, pad_width=[[0, block_count*CFG['block_size']-len(series)], [0, 0]])
    
    block_begins = list(range(0, len(series), CFG['block_stride']))
    block_begins = [x for x in block_begins if x+CFG['block_size'] <= len(series)]
    
    blocks = []
    for begin in block_begins:
        values = series[begin:begin + CFG['block_size']]
        blocks.append({'begin': begin,
                       'end': begin + ['block_size'],
                       'values': values})
    
    return blocks

# If using GPUs/TPUs on the Kaggle Server
GPU_BATCH_SIZE = 4
TPU_BATCH_SIZE = GPU_BATCH_SIZE * 8




## Models:

The models are a combination of transformer encoder, which you can read about [here](https://arxiv.org/pdf/1706.03762.pdf) and bi-directional LSTMs. The reasoning behind choosing this, based off EDTA and me trying simple models is as follows:
- Reason for LSTMs: Based of EDTA, its becomes evident that there is a clear spike in a certain value and that triggers a fog event. LSTMs made sense for this. In addition, it came to my attention that the event also was stopped at future point in the future of the reading, and hence why bidirection LSTMs were used. However, these are memory/computationally expensive, and had to rely on others for the code here.
- Limited the data, I did not use any of the subject data, or medication data. The community stated that this data had limited effect. In my simplier models, I tried combine a single directional LSTM with subject data, it had little impact.
- Transformer: this is the part of model I relied on help from others in the Kaggle community
>"a model architecture eschewing recurrence and instead relying entirely on an attention mechanism to draw global dependencies between input and output.
The Transformer allows for significantly more parallelization and can reach a new state of the art in
translation quality after being trained for as little as twelve hours on eight P100 GPUs." - [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf)
- No features: outside of normalization, bidirectional LSTMs are memory and computationally intensive, limiting the resolution is key here. In simpler models I created, I was not performing nearly as well.
- During training, there is slight randomize rolling of positional encoding was done here,as done by the [Baurzhan Urazalinov](https://www.kaggle.com/baurzhanurazalinov). I decided to keep it here.

### Tfog Model

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
    
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=CFG['fog_model_num_heads'], key_dim=CFG['fog_model_dim'], dropout=CFG['fog_model_mha_dropout'])
        
        self.add = tf.keras.layers.Add()
        
        self.layernorm = tf.keras.layers.LayerNormalization()
        
        self.seq = tf.keras.Sequential([tf.keras.layers.Dense(CFG['fog_model_dim'], activation='relu'), 
                                        tf.keras.layers.Dropout(CFG['fog_model_encoder_dropout']), 
                                        tf.keras.layers.Dense(CFG['fog_model_dim']), 
                                        tf.keras.layers.Dropout(CFG['fog_model_encoder_dropout']),
                                       ])
        
    def call(self, x):
        attn_output = self.mha(query=x, key=x, value=x)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        x = self.add([x, self.seq(x)])
        x = self.layernorm(x)
        
        return x
    
# FOGEncoder is a combination of transformer encoder (D=320, H=6, L=5) and two BidirectionalLSTM layers


class FOGEncoder(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
        self.first_linear = tf.keras.layers.Dense(CFG['fog_model_dim'])
        
        self.add = tf.keras.layers.Add()
        
        self.first_dropout = tf.keras.layers.Dropout(CFG['fog_model_first_dropout'])
        
        self.enc_layers = [EncoderLayer() for _ in range(CFG['fog_model_num_encoder_layers'])]
        
        self.lstm_layers = [tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(CFG['fog_model_dim'], return_sequences=True)) for _ in range(CFG['fog_model_num_lstm_layers'])]
        
        self.sequence_len = CFG['block_size'] // CFG['patch_size']
        self.pos_encoding = tf.Variable(initial_value=tf.random.normal(shape=(1, self.sequence_len, CFG['fog_model_dim']), stddev=0.02), trainable=True)
        
    def call(self, x, training=None): # (GPU_BATCH_SIZE, CFG['block_size'] // CFG['patch_size'], CFG['patch_size']*3), Example shape (4, 864, 54)
        x = x / 25.0 # Normalization attempt in the segment [-1, 1]
        x = self.first_linear(x) # (GPU_BATCH_SIZE, CFG['block_size'] // CFG['patch_size'], CFG['fog_model_dim']), Example shape (4, 864, 320)
          
        if training: # augmentation by randomly roll of the position encoding tensor
            random_pos_encoding = tf.roll(tf.tile(self.pos_encoding, multiples=[GPU_BATCH_SIZE, 1, 1]), 
                                          shift=tf.random.uniform(shape=(GPU_BATCH_SIZE,), minval=-self.sequence_len, maxval=0, dtype=tf.int32),
                                          axis=GPU_BATCH_SIZE * [1],
                                          )
            x = self.add([x, random_pos_encoding])
        
        else: # without augmentation 
            x = self.add([x, tf.tile(self.pos_encoding, multiples=[GPU_BATCH_SIZE, 1, 1])])
            
        x = self.first_dropout(x)
        
        for i in range(CFG['fog_model_num_encoder_layers']): x = self.enc_layers[i](x) # (GPU_BATCH_SIZE, CFG['block_size'] // CFG['patch_size'], CFG['fog_model_dim']), Example shape (4, 864, 320)
        for i in range(CFG['fog_model_num_lstm_layers']): x = self.lstm_layers[i](x) # (GPU_BATCH_SIZE, CFG['block_size'] // CFG['patch_size'], CFG['fog_model_dim']*2), Example shape (4, 864, 640)
            
        return x
    
class FOGModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
        self.encoder = FOGEncoder()
        self.last_linear = tf.keras.layers.Dense(3) 
        
    def call(self, x):
        x = self.encoder(x) 
        x = self.last_linear(x) 
        x = tf.nn.sigmoid(x)
        
        return x
    
WEIGHTS = '/kaggle/input/tfog-freezing-gait-models/016_0.557_0.893_0.0798_model.h5' # TDCSFOG weights
    
model = FOGModel()
model.build(input_shape=(GPU_BATCH_SIZE, CFG['block_size'] // CFG['patch_size'], CFG['patch_size']*3))
if len(WEIGHTS): model.load_weights(WEIGHTS)

### PredicitinFnCallback: Tfog

Here we use PredictionFnCallback. This allows us to:
1. Load test data
2. Model data preparation
3. Prediction

In [4]:
class PredictionFnCallback(tf.keras.callbacks.Callback):
    
    def __init__(self, prediction_ids, model=None, verbose=0):
        
        if not model is None: self.model = model
        self.verbose = verbose
         
        def init(Id, path):
            series = pd.read_csv(path).reset_index(drop=True)
            series['Id'] = Id
            series['AccV'] = sample_normalize(series['AccV'].values)
            series['AccML'] = sample_normalize(series['AccML'].values)
            series['AccAP'] = sample_normalize(series['AccAP'].values)
            
            series_blocks=[]
            for block in get_blocks(series, ['AccV', 'AccML', 'AccAP']): # Example shape (15552, 3)
                values = tf.reshape(block['values'], shape=(CFG['block_size'] // CFG['patch_size'], CFG['patch_size'], 3)) # Example shape (864, 18, 3)
                values = tf.reshape(values, shape=(CFG['block_size'] // CFG['patch_size'], CFG['patch_size']*3)) # Example shape (864, 54)
                values = tf.expand_dims(values, axis=0) # Example shape (1, 864, 54)
                
                self.blocks.append(values)
                series_blocks.append((self.blocks_counter, block['begin'], block['end']))
                self.blocks_counter += 1
            
            description = {}
            description['series'] = series
            description['series_blocks'] = series_blocks
            self.descriptions.append(description)
            
        self.descriptions = []
        self.blocks = [] 
        self.blocks_counter=0 
        
        tsfog_ids = prediction_ids
        tsfog_paths = [f'/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/tdcsfog/{tsfog_id}.csv' for tsfog_id in tsfog_ids]
        for tsfog_id, tsfog_path in tqdm(zip(tsfog_ids, tsfog_paths), total=len(tsfog_ids), desc='PredictionFnCallback Initialization', disable=1-verbose): 
            init(tsfog_id, tsfog_path)
            
        self.blocks = tf.concat(self.blocks, axis=0) # Example shape (self.blocks_counter, 864, 54)
        
        '''
        self.blocks is padded so that the final length is divisible by inference batch size for error-free operation of model.predict function
        Padded values have no effect on the predictions
        
        '''
        
        self.blocks = tf.pad(self.blocks, 
                             paddings=[[0, math.ceil(self.blocks_counter / (TPU_BATCH_SIZE if CFG['TPU'] else GPU_BATCH_SIZE))*(TPU_BATCH_SIZE if CFG['TPU'] else GPU_BATCH_SIZE)-self.blocks_counter], 
                                                    [0, 0], 
                                                    [0, 0],
                                      ]) # Example shape (self.blocks_counter+pad_value, 864, 54)
        
        print(f'\n[EventPredictionFnCallback Initialization] [Series] {len(self.descriptions)} [Blocks] {self.blocks_counter}\n')
    
    def prediction(self):
        predictions = model.predict(self.blocks, batch_size=TPU_BATCH_SIZE if CFG['TPU'] else GPU_BATCH_SIZE, verbose=self.verbose) # Example shape (self.blocks_counter+pad_value, 864, 3)
        predictions = tf.expand_dims(predictions, axis=-1) # Example shape (self.blocks_counter+pad_value, 864, 3, 1)
        predictions = tf.transpose(predictions, perm=[0, 1, 3, 2]) # Example shape (self.blocks_counter+pad_value, 864, 1, 3)
        predictions = tf.tile(predictions, multiples=[1, 1, CFG['patch_size'], 1]) # Example shape (self.blocks_counter+pad_value, 864, 18, 3)
        predictions = tf.reshape(predictions, shape=(predictions.shape[0], predictions.shape[1]*predictions.shape[2], 3)) # Example shape (self.blocks_counter+pad_value, 15552, 3)
        predictions = predictions.numpy()
        
        '''
        The following function aggregates predictions blocks and creates dataframes with StartHesitation_prediction, Turn_prediction, Walking_prediction columns.
        
        '''
        
        def create_target(description):
            series, series_blocks = description['series'].copy(), description['series_blocks']
            
            values = np.zeros((series_blocks[-1][2], 4))
            for series_block in series_blocks:
                i, begin, end = series_block
                values[begin:end, 0:3] += predictions[i]
                values[begin:end, 3] += 1

            values = values[:len(series)]
            
            series['StartHesitation_prediction'] = values[:,0] / values[:, 3]
            series['Turn_prediction'] = values[:, 1] / values[:, 3]
            series['Walking_prediction'] = values[:, 2] / values[:, 3]
            series['Prediction_count'] = values[:, 3]
            series['Event_prediction'] = series[['StartHesitation_prediction', 'Turn_prediction', 'Walking_prediction']].aggregate('max', axis=1)
            
            return series
            
        targets = Parallel(n_jobs=-1)(delayed(create_target)(self.descriptions[i]) for i in tqdm(range(len(self.descriptions)), disable=1-self.verbose))
        targets = pd.concat(targets)
        
        return targets


[EventPredictionFnCallback Initialization] [Series] 1 [Blocks] 1



### Predicitions: Tfog

Here we run PredictionFnCallback on the test set.

In [ ]:
for Id in tsfog_ids:
    targets = PredictionFnCallback(prediction_ids=[Id], model=model).prediction()
    submission = pd.DataFrame({'Id': (targets['Id'].values + '_' + targets['Time'].astype('str')).values,
                               'StartHesitation': targets['StartHesitation_prediction'].values,
                               'Turn': targets['Turn_prediction'].values,
                               'Walking': targets['Walking_prediction'].values,
                              })
    
    all_submissions.append(submission)

## Model: Defog
This is the same model design, but utilizes only the defog dataset. Please refer to the tfog description for more details.

In [5]:
CFG = {'TPU': 0,
       'block_size': 12096, 
       'block_stride': 12096//16,
       'patch_size': 14, 
       
       'fog_model_dim': 320,
       'fog_model_num_heads': 6,
       'fog_model_num_encoder_layers': 5,
       'fog_model_num_lstm_layers': 2,
       'fog_model_first_dropout': 0.1,
       'fog_model_encoder_dropout': 0.1,
       'fog_model_mha_dropout': 0.0,
      }

assert CFG['block_size'] % CFG['patch_size'] == 0
assert CFG['block_size'] % CFG['block_stride'] == 0

def sample_normalize(sample):
    mean = tf.math.reduce_mean(sample)
    std = tf.math.reduce_std(sample)
    sample = tf.math.divide_no_nan(sample-mean, std)
    
    return sample.numpy()

def get_blocks(series, columns):
    series = series.copy()
    series = series[columns]
    series = series.values
    series = series.astype(np.float32)
    
    block_count = math.ceil(len(series) / CFG['block_size'])
    
    series = np.pad(series, pad_width=[[0, block_count*CFG['block_size']-len(series)], [0, 0]])
    
    block_begins = list(range(0, len(series), CFG['block_stride']))
    block_begins = [x for x in block_begins if x+CFG['block_size'] <= len(series)]
    
    blocks = []
    for begin in block_begins:
        values = series[begin:begin+CFG['block_size']]
        blocks.append({'begin': begin,
                       'end': begin+CFG['block_size'],
                       'values': values})
    
    return blocks


GPU_BATCH_SIZE = 4
TPU_BATCH_SIZE = GPU_BATCH_SIZE*8

class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=CFG['fog_model_num_heads'], key_dim=CFG['fog_model_dim'], dropout=CFG['fog_model_mha_dropout'])
        
        self.add = tf.keras.layers.Add()
        
        self.layernorm = tf.keras.layers.LayerNormalization()
        
        self.seq = tf.keras.Sequential([tf.keras.layers.Dense(CFG['fog_model_dim'], activation='relu'), 
                                        tf.keras.layers.Dropout(CFG['fog_model_encoder_dropout']), 
                                        tf.keras.layers.Dense(CFG['fog_model_dim']), 
                                        tf.keras.layers.Dropout(CFG['fog_model_encoder_dropout']),
                                       ])
        
    def call(self, x):
        attn_output = self.mha(query=x, key=x, value=x)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        x = self.add([x, self.seq(x)])
        x = self.layernorm(x)
        
        return x
    
class FOGEncoder(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
        self.first_linear = tf.keras.layers.Dense(CFG['fog_model_dim'])
        
        self.add = tf.keras.layers.Add()
        
        self.first_dropout = tf.keras.layers.Dropout(CFG['fog_model_first_dropout'])
        
        self.enc_layers = [EncoderLayer() for _ in range(CFG['fog_model_num_encoder_layers'])]
        
        self.lstm_layers = [tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(CFG['fog_model_dim'], return_sequences=True)) for _ in range(CFG['fog_model_num_lstm_layers'])]
        
        self.sequence_len = CFG['block_size'] // CFG['patch_size']
        self.pos_encoding = tf.Variable(initial_value=tf.random.normal(shape=(1, self.sequence_len, CFG['fog_model_dim']), stddev=0.02), trainable=True)
        
    def call(self, x, training=None): 
        x = x / 50.0 
        x = self.first_linear(x) 
          
        if training: # augmentation by randomly roll of the position encoding tensor
            random_pos_encoding = tf.roll(tf.tile(self.pos_encoding, multiples=[GPU_BATCH_SIZE, 1, 1]), 
                                          shift=tf.random.uniform(shape=(GPU_BATCH_SIZE,), minval=-self.sequence_len, maxval=0, dtype=tf.int32),
                                          axis=GPU_BATCH_SIZE * [1],
                                          )
            x = self.add([x, random_pos_encoding])
        
        else: # without augmentation 
            x = self.add([x, tf.tile(self.pos_encoding, multiples=[GPU_BATCH_SIZE, 1, 1])])
            
        x = self.first_dropout(x)
        
        for i in range(CFG['fog_model_num_encoder_layers']): x = self.enc_layers[i](x) 
        for i in range(CFG['fog_model_num_lstm_layers']): x = self.lstm_layers[i](x) 
            
        return x
    
class FOGModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
        self.encoder = FOGEncoder()
        self.last_linear = tf.keras.layers.Dense(4) 
        
    def call(self, x): 
        x = self.encoder(x) 
        x = self.last_linear(x) 
        x = tf.nn.sigmoid(x)
        
        return x

WEIGHTS = '/kaggle/input/defog-freezing-gait-models/025_0.355_0.816_0.1436_model.h5' # DEFOG weights

model = FOGModel()
model.build(input_shape=(GPU_BATCH_SIZE, CFG['block_size'] // CFG['patch_size'], CFG['patch_size']*3))
if len(WEIGHTS): model.load_weights(WEIGHTS)

### PredicitionFnCallback and Predictions: Defog

In [6]:
class PredictionFnCallback(tf.keras.callbacks.Callback):
    
    def __init__(self, prediction_ids, model=None, verbose=0):
        
        if not model is None: self.model = model
        self.verbose = verbose
         
        def init(Id, path):
            series = pd.read_csv(path).reset_index(drop=True)
            series['Id'] = Id
            series['AccV'] = sample_normalize(series['AccV'].values)
            series['AccML'] = sample_normalize(series['AccML'].values)
            series['AccAP'] = sample_normalize(series['AccAP'].values)
            
            series_blocks=[]
            for block in get_blocks(series, ['AccV', 'AccML', 'AccAP']): 
                values = tf.reshape(block['values'], shape=(CFG['block_size'] // CFG['patch_size'], CFG['patch_size'], 3)) 
                values = tf.reshape(values, shape=(CFG['block_size'] // CFG['patch_size'], CFG['patch_size']*3)) 
                values = tf.expand_dims(values, axis=0) 
                
                self.blocks.append(values)
                series_blocks.append((self.blocks_counter, block['begin'], block['end']))
                self.blocks_counter += 1
            
            description = {}
            description['series'] = series
            description['series_blocks'] = series_blocks
            self.descriptions.append(description)
            
        self.descriptions = [] 
        self.blocks = [] 
        self.blocks_counter=0 
                
        defog_ids = prediction_ids
        defog_paths = [f'/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/defog/{defog_id}.csv' for defog_id in defog_ids]
        for defog_id, defog_path in tqdm(zip(defog_ids, defog_paths), total=len(defog_ids), desc='PredictionFnCallback Initialization', disable=1-verbose): 
            init(defog_id, defog_path)
                
        self.blocks = tf.concat(self.blocks, axis=0)  
        
        '''
        self.blocks is padded so that the final length is divisible by inference batch size for error-free operation of model.predict function
        Padded values have no effect on the predictions
        
        '''
        
        self.blocks = tf.pad(self.blocks, 
                             paddings=[[0, math.ceil(self.blocks_counter / (TPU_BATCH_SIZE if CFG['TPU'] else GPU_BATCH_SIZE))*(TPU_BATCH_SIZE if CFG['TPU'] else GPU_BATCH_SIZE)-self.blocks_counter], 
                                                    [0, 0], 
                                                    [0, 0],
                                      ]) 
        
        print(f'\n[PredictionFnCallback Initialization] [Series] {len(self.descriptions)} [Blocks] {self.blocks_counter}\n')
    
    def prediction(self):
        predictions = model.predict(self.blocks, batch_size=TPU_BATCH_SIZE if CFG['TPU'] else GPU_BATCH_SIZE, verbose=self.verbose) # Example shape (self.blocks_counter+pad_value, 864, 4)
        predictions = predictions[:, :, :3] 
        predictions = tf.expand_dims(predictions, axis=-1) 
        predictions = tf.transpose(predictions, perm=[0, 1, 3, 2]) 
        predictions = tf.tile(predictions, multiples=[1, 1, CFG['patch_size'], 1]) 
        predictions = tf.reshape(predictions, shape=(predictions.shape[0], predictions.shape[1]*predictions.shape[2], 3)) 
        predictions = predictions.numpy()
        
        '''
        The following function aggregates predictions blocks and creates dataframes with StartHesitation_prediction, Turn_prediction, Walking_prediction columns.
        
        '''
        
        def create_target(description):
            series, series_blocks = description['series'].copy(), description['series_blocks']
            
            values = np.zeros((series_blocks[-1][2], 4))
            for series_block in series_blocks:
                i, begin, end = series_block
                values[begin:end, 0:3] += predictions[i]
                values[begin:end, 3] += 1

            values = values[:len(series)]
            
            series['StartHesitation_prediction'] = values[:, 0] / values[:, 3]
            series['Turn_prediction'] = values[:, 1] / values[:, 3]
            series['Walking_prediction'] = values[:, 2] / values[:, 3]
            series['Prediction_count'] = values[:, 3]
            
            return series
            
        targets = Parallel(n_jobs=-1)(delayed(create_target)(self.descriptions[i]) for i in tqdm(range(len(self.descriptions)), disable=1-self.verbose))
        targets = pd.concat(targets).reset_index(drop=True)
        
        return targets


for Id in defog_ids:
    targets = PredictionFnCallback(prediction_ids=[Id], model=model).prediction()
    submission = pd.DataFrame({'Id': (targets['Id'].values + '_' + targets['Time'].astype('str')).values,
                               'StartHesitation': targets['StartHesitation_prediction'].values,
                               'Turn': targets['Turn_prediction'].values,
                               'Walking': targets['Walking_prediction'].values,
                              })
    
    all_submissions.append(submission)


[PredictionFnCallback Initialization] [Series] 1 [Blocks] 369



## Submission:

The dataframes of the predicted values for both datasets can be concated below. If you decided to create multiple models, this allows for the averaging of values for your final submission. Again, I can not thank [Baurzhan Urazalinov](https://www.kaggle.com/baurzhanurazalinov) for his help here enough. It was a great learning experience.

In [7]:
submission = pd.concat(all_submissions).reset_index(drop=True)
submission = submission.groupby('Id').agg('mean').reset_index()
submission.to_csv('submission.csv', index=False)

## Conclusion:

Working with this dataset was extremely challenging for me. The community helped quiet a bit and there experience helped me learn more about using the TensorFlow API.

The EDTA was quiet helpful in realizing some form of memory was necassary and I immediately thought of LSTMs. [Baurzhan Urazalinov](https://www.kaggle.com/baurzhanurazalinov) work on pointing out transforms was extremely helpful and made this possible.

The best solutions are at >0.5 at this time. 

Future steps:
- Hypertuning parameters
- Potentionally combining both datasets
- Using a TPU, which I was unable to do this iteration
- The notype dataset was never used, I am curious if people thought of incorporating this data in an unsupervised way
- Personally, learning more about tensorflow and different models being developed by Google Brain